In [1]:
import pandas as pd

In [2]:
tracking = pd.concat([pd.read_csv(f"../data/tracking_week_{i}.csv") for i in range(1,10)])

In [3]:
plays_frames = tracking.drop_duplicates(["gameId","playId","frameId"])[["gameId","playId","frameId","event"]]

In [4]:
plays_events = plays_frames.dropna(subset=["event"]).groupby(["gameId","playId"])["event"].unique()

In [5]:
plays_events

gameId      playId
2022090800  56              [pass_arrived, pass_outcome_caught, tackle]
            80                             [run, first_contact, tackle]
            101             [ball_snap, handoff, first_contact, tackle]
            122       [pass_arrived, pass_outcome_caught, first_cont...
            146             [ball_snap, handoff, first_contact, tackle]
                                            ...                        
2022110700  3658            [ball_snap, handoff, first_contact, tackle]
            3686            [ball_snap, handoff, first_contact, tackle]
            3707            [ball_snap, handoff, first_contact, tackle]
            3740            [ball_snap, handoff, first_contact, tackle]
            3787            [ball_snap, handoff, first_contact, tackle]
Name: event, Length: 12486, dtype: object

In [6]:
events_sequences_counts = plays_events.astype(str).value_counts()
events_sequences = pd.Series(events_sequences_counts[events_sequences_counts>1].index)
events_sequences = events_sequences[events_sequences.apply(lambda x: "pass_outcome_caught" not in x or ("pass_outcome_caught" in x and "pass_arrived" in x))]

In [7]:
valid_plays = plays_events[plays_events.astype(str).isin(events_sequences.values)]

In [8]:
valid_plays

gameId      playId
2022090800  56              [pass_arrived, pass_outcome_caught, tackle]
            80                             [run, first_contact, tackle]
            101             [ball_snap, handoff, first_contact, tackle]
            122       [pass_arrived, pass_outcome_caught, first_cont...
            146             [ball_snap, handoff, first_contact, tackle]
                                            ...                        
2022110700  3658            [ball_snap, handoff, first_contact, tackle]
            3686            [ball_snap, handoff, first_contact, tackle]
            3707            [ball_snap, handoff, first_contact, tackle]
            3740            [ball_snap, handoff, first_contact, tackle]
            3787            [ball_snap, handoff, first_contact, tackle]
Name: event, Length: 11670, dtype: object

In [9]:
plays_frames_valid = plays_frames.set_index(["gameId", "playId"])
plays_frames_valid = plays_frames_valid.loc[valid_plays.index]

In [10]:
plays_frames_valid

frameId         event
gameId     playId                       
2022090800 56            1           NaN
           56            2  pass_arrived
           56            3           NaN
           56            4           NaN
           56            5           NaN
...                    ...           ...
2022110700 3787         40        tackle
           3787         41           NaN
           3787         42           NaN
           3787         43           NaN
           3787         44           NaN

[505069 rows x 2 columns]

In [11]:
possible_last_event = ["tackle", "out_of_bounds", "touchdown", "fumble", "qb_slide", "safety"]
run_event = ["handoff", "run"]
ball_snap_event = ["ball_snap", "snap_direct", "autoevent_ballsnap"]

def get_ball_carrier_from_event(group, plays_events):
    events = plays_events.loc[(group["gameId"].iloc[0], group["playId"].iloc[0])]
    if any(element in events for element in run_event):
        is_run_event = group["event"].isin(run_event)
        group.loc[is_run_event, "ball_carrier"] = "ball_carrier"
        if any(element in events for element in possible_last_event):
            group.loc[group["event"].isin(possible_last_event), "ball_carrier"] = "ball_carrier"
        
        if is_run_event.idxmax() != 0:
            group.loc[is_run_event.idxmax()-1, "ball_carrier"] = "qb"
        
        if any(element in events for element in ball_snap_event):
            group.loc[group["event"].isin(ball_snap_event), "ball_carrier"] = "qb"
        
            valid_indices = group["ball_carrier"].notnull()
            start_index = valid_indices.idxmax()
            end_index = valid_indices[::-1].idxmax()
            group.loc[start_index:end_index, "ball_carrier"] = group.loc[start_index:end_index, "ball_carrier"].ffill().bfill()
        
        else:
            group["ball_carrier"] = group["ball_carrier"].bfill()
        
    elif "pass_arrived" in events:
        is_pass_event = group["event"]=="pass_arrived"
        group.loc[is_pass_event, "ball_carrier"] = "ball_carrier"
        if any(element in events for element in possible_last_event):
            group.loc[group["event"].isin(possible_last_event), "ball_carrier"] = "ball_carrier"
        
        valid_indices = group["ball_carrier"].notnull()
        start_index = valid_indices.idxmax()
        end_index = valid_indices[::-1].idxmax()
        group.loc[start_index:end_index, "ball_carrier"] = group.loc[start_index:end_index, "ball_carrier"].ffill().bfill()

    return group

In [12]:
plays_frames_valid["ball_carrier"] = None
plays_frames_valid.loc[(2022090800,101)].reset_index().groupby(["gameId","playId"]).apply(lambda x: get_ball_carrier_from_event(x, plays_events))

gameId  playId  frameId          event  ball_carrier
gameId     playId                                                             
2022090800 101    0   2022090800     101        1            NaN          None
                  1   2022090800     101        2            NaN          None
                  2   2022090800     101        3            NaN          None
                  3   2022090800     101        4            NaN          None
                  4   2022090800     101        5            NaN          None
                  5   2022090800     101        6      ball_snap            qb
                  6   2022090800     101        7            NaN            qb
                  7   2022090800     101        8            NaN            qb
                  8   2022090800     101        9            NaN            qb
                  9   2022090800     101       10            NaN            qb
                  10  2022090800     101       11            NaN            qb
                  11  2022090800     101       12            NaN            qb
                  12  2022090800     101       13            NaN            qb
                  13  2022090800     101       14            NaN            qb
                  14  2022090800     101       15            NaN            qb
                  15  2022090800     101       16            NaN            qb
                  16  2022090800     101       17            NaN            qb
                  17  2022090800     101       18            NaN            qb
                  18  2022090800     101       19        handoff  ball_carrier
                  19  2022090800     101       20            NaN  ball_carrier
                  20  2022090800     101       21            NaN  ball_carrier
                  21  2022090800     101       22            NaN  ball_carrier
                  22  2022090800     101       23            NaN  ball_carrier
                  23  2022090800     101       24            NaN  ball_carrier
                  24  2022090800     101       25            NaN  ball_carrier
                  25  2022090800     101       26            NaN  ball_carrier
                  26  2022090800     101       27            NaN  ball_carrier
                  27  2022090800     101       28            NaN  ball_carrier
                  28  2022090800     101       29            NaN  ball_carrier
                  29  2022090800     101       30            NaN  ball_carrier
                  30  2022090800     101       31            NaN  ball_carrier
                  31  2022090800     101       32            NaN  ball_carrier
                  32  2022090800     101       33            NaN  ball_carrier
                  33  2022090800     101       34            NaN  ball_carrier
                  34  2022090800     101       35            NaN  ball_carrier
                  35  2022090800     101       36            NaN  ball_carrier
                  36  2022090800     101       37            NaN  ball_carrier
                  37  2022090800     101       38  first_contact  ball_carrier
                  38  2022090800     101       39            NaN  ball_carrier
                  39  2022090800     101       40            NaN  ball_carrier
                  40  2022090800     101       41            NaN  ball_carrier
                  41  2022090800     101       42            NaN  ball_carrier
                  42  2022090800     101       43            NaN  ball_carrier
                  43  2022090800     101       44            NaN  ball_carrier
                  44  2022090800     101       45         tackle  ball_carrier
                  45  2022090800     101       46            NaN          None
                  46  2022090800     101       47            NaN          None
                  47  2022090800     101       48            NaN          None
                  48  2022090800     101       49            NaN          No

In [13]:
plays = pd.read_csv("../data/plays.csv")
players = pd.read_csv("../data/players.csv")

players_positions = tracking[["gameId", "playId","nflId"]].drop_duplicates().dropna().merge(players[["nflId", "position"]], on="nflId")
qb_players = players_positions[players_positions["position"]=="QB"].drop(columns="position").rename(columns={"nflId": "qbId"})

In [14]:
def _get_ball_carrier_id(x):
    if x["ball_carrier"]=="qb":
        res = x["qbId"]
    elif x["ball_carrier"]=="ball_carrier":
        res = x["ballCarrierId"]
    else:
        res = None
    return res

In [16]:
test_play = plays_frames_valid.loc[(2022090800,101)].reset_index().groupby(["gameId","playId"]).apply(
    lambda x: get_ball_carrier_from_event(x, plays_events)
).reset_index(drop=True).merge(
    plays[["gameId", "playId", "ballCarrierId", "defensiveTeam", "absoluteYardlineNumber", "yardsToGo"]], on=["gameId","playId"]
)
test_play = test_play.merge(qb_players, on=["gameId", "playId"])
test_play["ball_carrier_id"] = test_play.apply(_get_ball_carrier_id, axis=1)

In [17]:
test_play

,gameId,playId,frameId,event,ball_carrier,ballCarrierId,defensiveTeam,absoluteYardlineNumber,yardsToGo,qbId,ball_carrier_id
0,2022090800,101,1,NaN,None,47857,LA,72,10,46076.0,NaN
1,2022090800,101,2,NaN,None,47857,LA,72,10,46076.0,NaN
2,2022090800,101,3,NaN,None,47857,LA,72,10,46076.0,NaN
3,2022090800,101,4,NaN,None,47857,LA,72,10,46076.0,NaN
4,2022090800,101,5,NaN,None,47857,LA,72,10,46076.0,NaN
5,2022090800,101,6,ball_snap,qb,47857,LA,72,10,46076.0,46076.0
6,2022090800,101,7,NaN,qb,47857,LA,72,10,46076.0,46076.0
7,2022090800,101,8,NaN,qb,47857,LA,72,10,46076.0,46076.0
8,2022090800,101,9,NaN,qb,47857,LA,72,10,46076.0,46076.0
9,2022090800,101,10,NaN,qb,47857,LA,72,10,46076.0,46076.0


In [18]:
test_play_tracking = tracking[["gameId", "playId","nflId","frameId", "club","x","y", "playDirection"]].merge(test_play, how="inner", on=["gameId", "playId","frameId"])
test_play_tracking["is_defense"] = test_play_tracking["club"] == test_play_tracking["defensiveTeam"]
test_play_tracking["is_ball_carrying"] = test_play_tracking["nflId"] == test_play_tracking["ball_carrier_id"]

In [19]:
test_play_tracking

,gameId,playId,nflId,frameId,club,x,y,playDirection,event,ball_carrier,ballCarrierId,defensiveTeam,absoluteYardlineNumber,yardsToGo,qbId,ball_carrier_id,is_defense,is_ball_carrying
0,2022090800,101,35472.0,1,BUF,73.130000,27.640000,left,NaN,NaN,47857,LA,72,10,46076.0,NaN,False,False
1,2022090800,101,38577.0,1,LA,67.320000,29.830000,left,NaN,NaN,47857,LA,72,10,46076.0,NaN,True,False
2,2022090800,101,41239.0,1,LA,71.280000,26.410000,left,NaN,NaN,47857,LA,72,10,46076.0,NaN,True,False
3,2022090800,101,42392.0,1,BUF,72.500000,29.460000,left,NaN,NaN,47857,LA,72,10,46076.0,NaN,False,False
4,2022090800,101,42816.0,1,LA,64.670000,40.760000,left,NaN,NaN,47857,LA,72,10,46076.0,NaN,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,2022090800,101,48512.0,49,BUF,73.340000,39.510000,left,NaN,NaN,47857,LA,72,10,46076.0,NaN,False,False
1123,2022090800,101,52536.0,49,BUF,60.050000,34.310000,left,NaN,NaN,47857,LA,72,10,46076.0,NaN,False,False
1124,2022090800,101,53079.0,49,BUF,66.780000,46.350000,left,NaN,NaN,47857,LA,72,10,46076.0,NaN,False,False
1125,2022090800,101,53522.0,49,BUF,67.180000,44.930000,left,NaN,NaN,47857,LA,72,10,46076.0,NaN,False,False


In [20]:
test_play_tracking.to_csv("../data/test_play_tracking.csv", index=False)